# Exemplo Completo: Classificação de Vegetação da Caatinga

Este notebook demonstra o uso do sistema integrado para classificação de tipos funcionais de vegetação da Caatinga usando dados de satélites hiperespectrais (PRISMA, EnMAP, EMIT).

## 1. Importações e Configuração

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Importar o módulo de classificação
from caatinga_classification import (
    CaatingaPipeline, 
    SpectralConfig,
    SatelliteDataLoader,
    SpectralPreprocessor,
    EndmemberExtractor,
    CaatingaClassifier
)

# Configurar matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Importações concluídas")

## 2. Configuração do Pipeline

In [ ]:
# Criar configuração personalizada
config = SpectralConfig(
    savgol_window=11,          # Janela do filtro Savitzky-Golay
    savgol_polyorder=3,        # Ordem do polinômio
    n_components_pca=10,       # Componentes principais
    n_endmembers=5,            # Número de endmembers a extrair
    dbscan_eps=0.3,            # Epsilon para clustering DBSCAN
    dbscan_min_samples=5,      # Amostras mínimas para DBSCAN
    min_valid_pixels=100,      # Mínimo de pixels válidos
    max_cloud_cover=0.3        # Cobertura máxima de nuvens (30%)
)

# Inicializar o pipeline
pipeline = CaatingaPipeline(config)

print("✓ Pipeline configurado")
print(f"  - Número de endmembers: {config.n_endmembers}")
print(f"  - Filtro Savitzky-Golay: janela={config.savgol_window}, ordem={config.savgol_polyorder}")

## 3. Definir Caminhos dos Dados

**IMPORTANTE**: Ajuste os caminhos abaixo para seus dados reais.

In [ ]:
# ============================================
# AJUSTE ESTES CAMINHOS PARA SEUS DADOS
# ============================================

# Diretório raiz dos dados
data_directory = '/caminho/para/seus/dados/satelites'

# Exemplos de caminhos para cada sensor
enmap_scene_path = '/caminho/para/cena/ENMAP_L1B/pasta_da_cena'
emit_scene_path = '/caminho/para/cena/EMIT_L2A_reflectancia.tif'
prisma_scene_path = '/caminho/para/cena/PRISMA_L1_data.h5'

# Diretório de saída
output_directory = './resultados_classificacao'
Path(output_directory).mkdir(parents=True, exist_ok=True)

print("✓ Caminhos configurados")
print(f"  - Diretório de dados: {data_directory}")
print(f"  - Diretório de saída: {output_directory}")

## 4. Processar Cena EnMAP

Processa uma cena EnMAP completa (VNIR + SWIR).

In [ ]:
# Definir diretório de dados
pipeline.set_data_directory(data_directory)

# Processar cena EnMAP
print("Processando cena EnMAP...\n")
results_enmap = pipeline.process_scene(
    scene_path=enmap_scene_path,
    sensor='ENMAP'
)

print("\n✓ Processamento EnMAP concluído")

## 5. Visualizar Endmembers Extraídos

In [ ]:
# Plotar endmembers
endmembers = results_enmap['endmembers']
n_bands = endmembers.shape[1]

plt.figure(figsize=(12, 6))
for i, endmember in enumerate(endmembers):
    plt.plot(range(n_bands), endmember, label=f'Endmember {i+1}', linewidth=2, alpha=0.8)

plt.title('Endmembers Extraídos - EnMAP', fontsize=14, fontweight='bold')
plt.xlabel('Número da Banda', fontsize=12)
plt.ylabel('Reflectância Normalizada', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{output_directory}/endmembers_enmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figura salva em {output_directory}/endmembers_enmap.png")

## 6. Análise de Índices Espectrais

In [ ]:
# Extrair índices espectrais
indices = results_enmap['spectral_indices']

# Criar subplots para cada índice
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Distribuição de Índices Espectrais - EnMAP', fontsize=16, fontweight='bold')

index_names = ['NDVI', 'SAVI', 'EVI', 'NDWI', 'BSI']
colors = ['green', 'olive', 'darkgreen', 'blue', 'brown']

for idx, (name, color) in enumerate(zip(index_names, colors)):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    values = indices[name]
    
    # Histograma
    ax.hist(values, bins=50, color=color, alpha=0.7, edgecolor='black')
    
    # Estatísticas
    mean_val = np.mean(values)
    std_val = np.std(values)
    median_val = np.median(values)
    
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Média: {mean_val:.3f}')
    ax.axvline(median_val, color='blue', linestyle=':', linewidth=2, label=f'Mediana: {median_val:.3f}')
    
    ax.set_title(f'{name}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Valor', fontsize=10)
    ax.set_ylabel('Frequência', fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

# Remover subplot vazio
axes[1, 2].axis('off')

plt.tight_layout()
plt.savefig(f'{output_directory}/indices_espectrais_enmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figura salva em {output_directory}/indices_espectrais_enmap.png")

## 7. Distribuição de Classes de Vegetação

In [ ]:
# Obter classes
classes = results_enmap['classes']
class_names = pipeline.classifier.CAATINGA_TYPES

# Contar classes
unique_classes, counts = np.unique(classes, return_counts=True)
percentages = (counts / len(classes)) * 100

# Criar DataFrame
df_classes = pd.DataFrame({
    'Classe': [class_names[c] for c in unique_classes],
    'Pixels': counts,
    'Percentual (%)': percentages
})

print("\nDistribuição de Classes:")
print("="*50)
print(df_classes.to_string(index=False))
print("="*50)

# Gráfico de barras
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Distribuição de Classes de Vegetação - EnMAP', fontsize=14, fontweight='bold')

# Gráfico de barras
colors_bar = ['darkgreen', 'green', 'olive', 'yellowgreen', 'yellow', 'brown']
ax1.bar([class_names[c] for c in unique_classes], counts, color=colors_bar[:len(unique_classes)])
ax1.set_xlabel('Tipo de Vegetação', fontsize=12)
ax1.set_ylabel('Número de Pixels', fontsize=12)
ax1.set_title('Contagem de Pixels por Classe', fontsize=12)
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# Gráfico de pizza
ax2.pie(percentages, labels=[class_names[c] for c in unique_classes], 
        autopct='%1.1f%%', colors=colors_bar[:len(unique_classes)],
        startangle=90)
ax2.set_title('Distribuição Percentual', fontsize=12)

plt.tight_layout()
plt.savefig(f'{output_directory}/distribuicao_classes_enmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Figura salva em {output_directory}/distribuicao_classes_enmap.png")

## 8. Visualização Completa de Resultados

In [ ]:
# Usar a função integrada de visualização
pipeline.visualize_results(
    results_enmap,
    output_dir=output_directory
)

print("✓ Visualização completa gerada")

## 9. Exportar Resultados

In [ ]:
# Exportar para CSV
output_csv = f'{output_directory}/classificacao_enmap.csv'
pipeline.export_results(results_enmap, output_csv)

# Carregar e exibir primeiras linhas
df_results = pd.read_csv(output_csv)
print("\nPrimeiras 10 linhas dos resultados:")
print(df_results.head(10))

print(f"\n✓ Resultados completos salvos em {output_csv}")

## 10. Processar Cena EMIT (Opcional)

In [ ]:
# Descomentar para processar EMIT
# print("Processando cena EMIT...\n")
# results_emit = pipeline.process_scene(
#     scene_path=emit_scene_path,
#     sensor='EMIT'
# )
# 
# pipeline.visualize_results(results_emit, output_dir=f'{output_directory}/emit')
# pipeline.export_results(results_emit, f'{output_directory}/classificacao_emit.csv')
# 
# print("✓ Processamento EMIT concluído")

## 11. Processar Cena PRISMA (Opcional)

In [ ]:
# Descomentar para processar PRISMA
# print("Processando cena PRISMA...\n")
# results_prisma = pipeline.process_scene(
#     scene_path=prisma_scene_path,
#     sensor='PRISMA'
# )
# 
# pipeline.visualize_results(results_prisma, output_dir=f'{output_directory}/prisma')
# pipeline.export_results(results_prisma, f'{output_directory}/classificacao_prisma.csv')
# 
# print("✓ Processamento PRISMA concluído")

## 12. Análise Comparativa (Se múltiplos sensores processados)

In [ ]:
# Exemplo de comparação entre sensores
# if 'results_emit' in locals() and 'results_prisma' in locals():
#     # Comparar NDVI médio
#     ndvi_enmap = np.mean(results_enmap['spectral_indices']['NDVI'])
#     ndvi_emit = np.mean(results_emit['spectral_indices']['NDVI'])
#     ndvi_prisma = np.mean(results_prisma['spectral_indices']['NDVI'])
#     
#     print("\nComparação de NDVI Médio:")
#     print(f"  EnMAP:  {ndvi_enmap:.3f}")
#     print(f"  EMIT:   {ndvi_emit:.3f}")
#     print(f"  PRISMA: {ndvi_prisma:.3f}")
#     
#     # Plotar comparação
#     sensors = ['EnMAP', 'EMIT', 'PRISMA']
#     ndvi_values = [ndvi_enmap, ndvi_emit, ndvi_prisma]
#     
#     plt.figure(figsize=(10, 6))
#     plt.bar(sensors, ndvi_values, color=['green', 'blue', 'red'])
#     plt.title('Comparação de NDVI Médio entre Sensores', fontsize=14, fontweight='bold')
#     plt.xlabel('Sensor', fontsize=12)
#     plt.ylabel('NDVI Médio', fontsize=12)
#     plt.grid(True, alpha=0.3, axis='y')
#     plt.tight_layout()
#     plt.savefig(f'{output_directory}/comparacao_sensores.png', dpi=300, bbox_inches='tight')
#     plt.show()

## Conclusão

Este notebook demonstrou:
1. Configuração do pipeline de classificação
2. Carregamento e processamento de dados EnMAP
3. Extração de endmembers usando AEEB
4. Cálculo de índices espectrais
5. Classificação automática de tipos de vegetação
6. Visualização e exportação de resultados

O mesmo processo pode ser aplicado a dados EMIT e PRISMA, permitindo análise multi-sensor da vegetação da Caatinga.